In [651]:
import pandas as pd 
import altair as alt
import datetime as DT
from rating_parse import parse_ratings
import geopandas as gpd

In [652]:
df = pd.read_csv("./rating_parse/reviews.csv")
df.head()

,Unnamed: 0,review_id,title,date,country,author,author_url,version,rating,review,vote_count,page
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [653]:
df = df.dropna().reset_index()

In [654]:
df.head()

,index,Unnamed: 0,review_id,title,date,country,author,author_url,version,rating,review,vote_count,page
0,8,8,8.130477e+09,Lo compré hace tiempo y aún siguen actualizando,2021-12-14T13:55:57-07:00,ARG,Manumanolo,https://itunes.apple.com/ar/reviews/id172571332,5.0.0,5.0,Mucha calidad y gentileza de los programadores...,0.0,1.0
1,9,9,7.907065e+09,Elimina líneas,2021-10-12T09:08:48-07:00,ARG,fm_vm,https://itunes.apple.com/ar/reviews/id250902838,4.4.10,5.0,La mejor app para eliminar líneas como cables....,0.0,1.0
2,10,10,7.568497e+09,No compre este producto,2021-07-12T08:54:21-07:00,ARG,isabtzka,https://itunes.apple.com/ar/reviews/id367764749,4.4.10,1.0,No compre este producto y me lo cargaron a u$1...,0.0,1.0
3,11,11,6.921974e+09,No compré este producto,2021-01-27T19:39:16-07:00,ARG,veru28,https://itunes.apple.com/ar/reviews/id231140086,4.4.8,1.0,No compré este producto y me lo cargaron a u$ ...,1.0,1.0
4,12,12,6.554649e+09,Excellent!!,2020-10-20T04:02:06-07:00,ARG,Mardelmar2006,https://itunes.apple.com/ar/reviews/id461415498,4.4.8,5.0,"La app funciona muy bien, sobre todo para los ...",1.0,1.0


In [655]:
df = df[["date", "country", "rating"]]

In [656]:
df['date'] = df.date.apply(lambda x: x.split('T')[0])

In [657]:
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

In [658]:
df.head()

,date,country,rating
0,2021-12-14,ARG,5.0
1,2021-10-12,ARG,5.0
2,2021-07-12,ARG,1.0
3,2021-01-27,ARG,1.0
4,2020-10-20,ARG,5.0


In [659]:
year_ago = DT.date.today() - DT.timedelta(days=731)

In [660]:
# df_filtered = df.loc[df['date'] > pd.to_datetime(year_ago)]

In [661]:
df

,date,country,rating
0,2021-12-14,ARG,5.0
1,2021-10-12,ARG,5.0
2,2021-07-12,ARG,1.0
3,2021-01-27,ARG,1.0
4,2020-10-20,ARG,5.0
...,...,...,...
4870,2019-05-23,VNM,5.0
4871,2018-07-09,VNM,5.0
4872,2018-06-08,VNM,5.0
4873,2017-03-23,VNM,1.0


In [662]:
world = gpd.read_file("./data/world-countries.json")

In [663]:
world.head()

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [664]:
# alt.Chart(df_filtered).mark_line(

# ).transform_window(
#     sort=[{'field':"date"}],
#     mean_rating = 'mean(rating)',
#     frame=[None,0],
#     groupby=['country']
# ).encode(
#     x=alt.X("yearmonth(date):T"),
#     y=alt.Y("mean(mean_rating):Q", scale=alt.Scale(domain=[1, 5.0])),
#     color=alt.Color("country"),
#     tooltip = alt.Tooltip("country")
# ).transform_filter(
#     alt.FieldOneOfPredicate(field='country', oneOf=[ 'USA', 'GBR', 'DEU', 'JPN', "RUS", "KOR"]) 
# ).properties(width=800, height=600)

In [665]:
alt.Chart(df
).transform_filter(
       alt.FieldOneOfPredicate(field='country', oneOf=[ 'USA', 'GBR', 'DEU', 'JPN', "RUS", "KOR"]) 
).transform_density(
    'rating',
    groupby=['country'],
    as_=['rating', 'density'],
    
).mark_area().encode(
    x="rating:Q",
    y="density:Q",
    
).facet(
    "country:N",
    columns =3
)

alt.FacetChart(...)

In [666]:
df["year/week"] = df['date'].apply(lambda x: "%d-%d"%(x.week,x.year))

In [667]:
grouped = df.groupby(['year/week', 'rating']).size()

In [668]:
grouped.index

MultiIndex([('1-2011', 1.0),
            ('1-2011', 4.0),
            ('1-2011', 5.0),
            ('1-2012', 1.0),
            ('1-2012', 3.0),
            ('1-2012', 4.0),
            ('1-2012', 5.0),
            ('1-2013', 2.0),
            ('1-2013', 5.0),
            ('1-2014', 1.0),
            ...
            ('9-2019', 2.0),
            ('9-2019', 5.0),
            ('9-2020', 1.0),
            ('9-2020', 2.0),
            ('9-2020', 3.0),
            ('9-2020', 4.0),
            ('9-2020', 5.0),
            ('9-2021', 1.0),
            ('9-2021', 4.0),
            ('9-2021', 5.0)],
           names=['year/week', 'rating'], length=1410)

In [669]:
grouped = grouped.unstack(level=1).fillna(0).reset_index()

In [670]:
grouped['year/week'] = grouped['year/week'].apply(lambda x: "0-" + x)

In [671]:
grouped

rating,year/week,1.0,2.0,3.0,4.0,5.0
0,0-1-2011,1.0,0.0,0.0,2.0,2.0
1,0-1-2012,1.0,0.0,2.0,2.0,6.0
2,0-1-2013,0.0,1.0,0.0,0.0,1.0
3,0-1-2014,1.0,1.0,0.0,0.0,1.0
4,0-1-2015,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
549,0-9-2016,1.0,0.0,2.0,0.0,0.0
550,0-9-2018,5.0,0.0,0.0,0.0,2.0
551,0-9-2019,3.0,2.0,0.0,0.0,6.0
552,0-9-2020,10.0,2.0,1.0,1.0,13.0


In [672]:
grouped['year/week'] = pd.to_datetime(grouped['year/week'], format="%w-%W-%Y")

In [673]:
grouped

rating,year/week,1.0,2.0,3.0,4.0,5.0
0,2011-01-09,1.0,0.0,0.0,2.0,2.0
1,2012-01-08,1.0,0.0,2.0,2.0,6.0
2,2013-01-13,0.0,1.0,0.0,0.0,1.0
3,2014-01-12,1.0,1.0,0.0,0.0,1.0
4,2015-01-11,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
549,2016-03-06,1.0,0.0,2.0,0.0,0.0
550,2018-03-04,5.0,0.0,0.0,0.0,2.0
551,2019-03-10,3.0,2.0,0.0,0.0,6.0
552,2020-03-08,10.0,2.0,1.0,1.0,13.0


In [674]:
grouped['positive_reviews'] = grouped[5.0] + grouped[4.0]

In [675]:
grouped['negative_reviews'] = grouped[3.0] + grouped[2.0] + grouped[1.0]

In [676]:
grouped = grouped.rename({"year/week": "date", 1:"1", 2:"2", 3:"3", 4:"4", 5:"5"}, axis=1)
grouped.head()

rating,date,1,2,3,4,5,positive_reviews,negative_reviews
0,2011-01-09,1.0,0.0,0.0,2.0,2.0,4.0,1.0
1,2012-01-08,1.0,0.0,2.0,2.0,6.0,8.0,3.0
2,2013-01-13,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,2014-01-12,1.0,1.0,0.0,0.0,1.0,1.0,2.0
4,2015-01-11,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [677]:
# grouped = grouped.loc[grouped["date"].dt.year > 2020]

In [678]:
grouped['week'] = grouped['date'].apply(lambda x: x.week)

In [679]:
grouped['diff'] = grouped["positive_reviews"] - grouped['negative_reviews']

In [680]:
grouped.head()

rating,date,1,2,3,4,5,positive_reviews,negative_reviews,week,diff
0,2011-01-09,1.0,0.0,0.0,2.0,2.0,4.0,1.0,1,3.0
1,2012-01-08,1.0,0.0,2.0,2.0,6.0,8.0,3.0,1,5.0
2,2013-01-13,0.0,1.0,0.0,0.0,1.0,1.0,1.0,2,0.0
3,2014-01-12,1.0,1.0,0.0,0.0,1.0,1.0,2.0,2,-1.0
4,2015-01-11,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2,-1.0


In [681]:
input_slider = alt.binding_range(min=2010, max=2021, step=1, name='select year:')
select_year = alt.selection_single(name="year", fields = ['year'], bind=input_slider, init = {'year': 2017})

# **Коментарі**
У цьому проекті я вирішив спробувати візуалізувати рейтинги аплікаї на аппсторі для отримання інсайту про те, куди прямує її репутація, та як вона змінювалась з часом. Так як в інтернеті підходящих даних не було, я вручну запарсив дані через API Apple та підготував їх для використання

In [710]:
bars = alt.Chart(grouped

).mark_bar(

).encode(
    x=alt.X("week:Q", 
    axis=alt.Axis(tickCount=54, tickColor="grey", gridOpacity=0.5, labelOpacity=0.5)),
    
    y=alt.Y("diff:Q", 
    scale=alt.Scale(domain=[int(grouped["diff"].min()*1.1), int(grouped["diff"].max()*1.1)]),
    axis=alt.Axis( tickColor="grey", gridOpacity=0.5, labelOpacity=0.5)),
    
    color = alt.condition(
        alt.datum.diff > 0,
        alt.value("lightgreen"),
        alt.value("black")
    ),
    tooltip="date"
).add_selection(select_year).transform_filter(f'year(datum.date) == {select_year.year}')
line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='grey').encode(y='y')
bar_diff = (bars + line).properties(width=1200, height=600, title="Positive reviews over negative ones").interactive(bind_x=False)
# bar_diff

In [711]:
rating = alt.Chart(df).mark_area(

color = 'lightgreen',
opacity=0.5,
line ={'color':'black'}

).encode(
    x=alt.X("month(date):T", axis=alt.Axis(title='month',tickColor="grey", gridOpacity=0.5, labelOpacity=0.5)),
    y=alt.Y("mean(rating):Q", scale=alt.Scale(domain=[1, 5.0]), axis=alt.Axis( tickColor="grey", gridOpacity=0.5, labelOpacity=0.5)),
    color=alt.value("lightgreen"),
    # color="country",
    tooltip = alt.Tooltip("count()"),
    
).add_selection(select_year
).transform_filter(
    f'year(datum.date) == {select_year.year}'

).properties(width=1200, height=600, title="Change in rating over months")

In [712]:
alt.vconcat(bar_diff, rating)

alt.VConcatChart(...)

# **Коментарі**
Основна мета цього чарту -- візуалізувати зміну перфомансу аплікації в плані кількості позитивних/негативних відгуків в розрізі тижнів. Для цього я використав бар чарт з відʼємними значеннями, так як він одразу демонструє хороший чи поганий результат напрямом палки і також скейл результату її довжиною, для зручності додано можливість скролом масштабувати графік по y шкалі. Також до цього чарту додається чарт, який демонструє зміну рейтингу апки в ті самі періоди часу, таким чином користувач бачить, як впливають негативні та позитивні відгуки на загальний середній рейтинг.

In [685]:
select_region = alt.selection_single(on = 'mouseover', fields = ['country'],name="country", nearest = False, empty = 'none')

In [699]:
lines = alt.Chart(df).mark_line(

).transform_window(
    sort=[{'field':"date"}],
    mean_rating = 'mean(rating)',
    frame=[31,0],
    groupby=['country']
).encode(
    x=alt.X("yearmonth(date):T",axis=alt.Axis(title='month',tickColor="grey", gridOpacity=0.5, labelOpacity=0.5)),
    y=alt.Y("mean(mean_rating):Q",axis=alt.Axis(title='Mean rating',tickColor="grey", gridOpacity=0.5, labelOpacity=0.5), scale=alt.Scale(domain=[1, 5.0])),
    color = alt.Color('country:N', legend=None), 
    strokeWidth = alt.condition(
    select_region,
    alt.value(5),
    alt.value(1)
    ),
    opacity=alt.condition(
        select_region,
        alt.value(1),
        alt.value(0.7)
    ),
    tooltip = alt.Tooltip("country:N")
).add_selection(select_region
).add_selection(select_year
).transform_filter(
    alt.FieldOneOfPredicate(field='country', oneOf=[ 'USA', 'GBR', 'DEU', 'JPN', "RUS", "AUS"]) 
).transform_filter(f'year(datum.date) =={select_year.year}'
).properties(width=800, height=600, title="Change in rating in main countries over a year")

In [704]:
dens = alt.Chart(df
).add_selection(select_year
).transform_filter(f'year(datum.date) =={select_year.year}'
).transform_density(
    'rating',
    groupby=['country'],
    as_=['rating', 'density'],
    
).mark_area(opacity=0.5).encode(
    x=alt.X("rating:Q",axis=alt.Axis(title='rating',tickColor="grey", gridOpacity=0.5, labelOpacity=0.5)),
    y=alt.Y("density:Q",axis=alt.Axis(title='density',tickColor="grey", gridOpacity=0.5, labelOpacity=0.5)),
    color=alt.value('darkgreen'),
).add_selection(select_region  

).transform_filter(
    select_region
).properties(width=400, height=600, title="Starr distribution")


In [705]:
alt.vconcat(alt.hconcat(lines , dens)).configure_view(
    stroke=None).configure_concat(
    spacing=1
)


alt.VConcatChart(...)

# **Коментарі**
Основна мета цього чарту -- дати користувачеві можливість порівняти, як змінюється середній рейтинг впродовж року на різних ключових ринках. Для цього я використав лінійний чарт, та покращив сприйняття через підсвічення та потовщення лінії, коли користувач наводить мишою на певну лінію. В додачу до цього чарту, я додав графік, який показує загальний розподіл балів відгуків у цій країні та у цей рік